In [1]:
from Utils.PySqlite import *
from Utils.Metrics import *
from myconfig import *
from Utils.Preprocess import *
from Utils.Apriori import CustomApriori
from m3ar import M3arAlgorithm
from mm3ar import MM3arAlgorithm
from otherAlgo import OtherAlgorithm
import pandas as pd
import csv
import matplotlib.pyplot as plt
from prettyprinter import cpprint as pp
import pickle
# from apyori import apriori
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

ALGO = 'mm3ar'


In [2]:
if not os.path.isfile(DATABASE_PATH): # create SQL table
    conn = CreateConnection(DATABASE_PATH)
    with conn:
        CreateTable(conn)

# if os.path.isfile(PRE_DATA_PATH):
#     os.remove(PRE_DATA_PATH)
# if os.path.isfile(R_INIT_PATH):
#     os.remove(R_INIT_PATH)

In [3]:
if not os.path.isfile(PRE_DATA_PATH):
    os.makedirs(os.path.dirname(PRE_DATA_PATH), exist_ok=True)
    df = Preprocess(INPUT_DATA_PATH, PRE_DATA_PATH, DATA_SAMPLE)

df = pd.read_csv(PRE_DATA_PATH, names=USE_COLUMNS,
                 index_col=False, skipinitialspace=True)

# df.head(20)


Total row: 30162


In [4]:
# _, R_inital, R_inital_lower = CustomApriori(df, MIN_SUP, MIN_CONF,True)
# os.makedirs(os.path.dirname(R_INIT_PATH), exist_ok=True)
# with open(R_INIT_PATH+'.csv', 'w', newline='') as f:
#     write = csv.writer(f)
#     write.writerow(['lhs','rhs','sup','conf','ruleStr','supAB','supA'])
#     write.writerows(R_inital)

# print("Total rule initial:", len(R_inital))
# pp(R_inital[0:2])
# print("Total rule initial lower:", len(R_inital_lower))

if os.path.isfile(R_INIT_PATH+'.data'):
    with open(R_INIT_PATH+'.data', 'rb') as f:  # load saved RInitial
        R_inital = pickle.load(f)
    with open(R_INIT_PATH+'-low.data', 'rb') as f:  # load saved RInitial
        R_inital_lower = pickle.load(f)
else:
    _, R_inital, R_inital_lower = CustomApriori(df, MIN_SUP, MIN_CONF, True)
    os.makedirs(os.path.dirname(R_INIT_PATH), exist_ok=True)
    file = open(R_INIT_PATH+'.data', 'wb')
    pickle.dump(R_inital, file)
    file.close()

    file = open(R_INIT_PATH + '-low.data', 'wb')
    pickle.dump(R_inital_lower, file)
    file.close()

with open(R_INIT_PATH+'.csv', 'w', newline='') as f:
    write = csv.writer(f)
    write.writerow(['lhs', 'rhs', 'sup', 'conf', 'ruleStr', 'supAB', 'supA'])
    write.writerows(R_inital)

print("Total rule initial:", len(R_inital))
pp(R_inital[0:10])

print("Total rule initial lower:", len(R_inital_lower))


Total rule initial: 150
[
    [
        {'sex|Male'},
        {'capital-loss|0'},
        0.6395464491744579,
        0.9465161923454367,
        'sex|Male->capital-loss|0',
        19290,
        20380
    ],
    [
        {'capital-loss|0'},
        {'sex|Male'},
        0.6395464491744579,
        0.6713067687489125,
        'capital-loss|0->sex|Male',
        19290,
        28735
    ],
    [
        {'race|White'},
        {'sex|Male'},
        0.5980372654333267,
        0.6955616396097636,
        'race|White->sex|Male',
        18038,
        25933
    ],
    [
        {'sex|Male'},
        {'race|White'},
        0.5980372654333267,
        0.8850834151128557,
        'sex|Male->race|White',
        18038,
        20380
    ],
    [
        {'native-country|United-States'},
        {'capital-gain|0'},
        0.8336648763344606,
        0.9142306573589296,
        'native-country|United-States->capital-gain|0',
        25145,
        27504
    ],
    [
        {'capital-gain|0

In [5]:
# for k in [5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 35, 40, 45,  50, 55, 60, 65, 70]:
for k in range(7,70,4):

    print('---------------------------')
    print("K_ANONYMITY=", k)
    anonymityRs = None
    if ALGO == 'm3ar':
        anonymity = M3arAlgorithm()
        anonymityRs = anonymity.Anonymize(df=df, RInitial=R_inital, k=k, algo=ALGO)
    elif ALGO == 'mm3ar':
        anonymity = MM3arAlgorithm()
        anonymityRs = anonymity.Anonymize(df=df, RInitial=R_inital, RInitalLow=R_inital_lower, k=k, algo=ALGO)
    else:
        anonymity = OtherAlgorithm()
        anonymityRs = anonymity.Anonymize(RInitial=R_inital, k=k, algo=ALGO)

    groups = anonymityRs['groups']

    dfAnonymity = pd.read_csv(KANONYMITY_DATA_PATH.format(ALGO, k), names=USE_COLUMNS,
                              index_col=False, skipinitialspace=True)
    _, RInitalNew, _ = CustomApriori(dfAnonymity, MIN_SUP, MIN_CONF)
    print('RInitalNew:', len(RInitalNew))
    # pp(RInitalNew[0:5])
    with open(R_INIT_PATH+'-new.csv', 'w', newline='') as f:
        write = csv.writer(f)
        write.writerow(['lhs', 'rhs', 'sup', 'conf', 'ruleStr', 'supAB', 'supA'])
        write.writerows(RInitalNew)

    # oldRules = [x[4] for x in R_inital]
    # newRules = [x[4] for x in RInitalNew]
    # oldDffRules = set(oldRules) - set(newRules)
    # pp(oldDffRules)

    lrp = LRP(R_inital, RInitalNew)
    nrp = NRP(R_inital, RInitalNew)
    drp = DRP(R_inital, RInitalNew)
    cavg = CAVG(groups, k)
    print('LRP: {} NRP: {} DRP:{} CAVG: {}'.format(lrp, nrp, drp, cavg))

    conn = CreateConnection(DATABASE_PATH)
    with conn:
        # algo,k,total,time_exe,
        # sg_bf,ug_bf,um,sg_af,ug_af,rules_bf,rules_af,
        # lrp, nrp, drp, cavg
        _ = InsertRecord(conn, (ALGO, k, df.shape[0], anonymityRs['totalTime'],
                                anonymityRs['lenOfGroupsBefore']['SG'],
                                anonymityRs['lenOfGroupsBefore']['UG'],
                                anonymityRs['lenOfGroupsUM']['UM'],
                                anonymityRs['lenOfGroupsAfter']['SG'],
                                anonymityRs['lenOfGroupsAfter']['UG'],
                                len(R_inital), len(RInitalNew),
                                lrp, nrp, drp, cavg))

    f = open("data/eval_result.txt", "a")
    with f:
        f.writelines(['{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'.format(ALGO, k, df.shape[0], anonymityRs['totalTime'],
                                                                              anonymityRs['lenOfGroupsBefore']['SG'],
                                                                              anonymityRs['lenOfGroupsBefore']['UG'],
                                                                              anonymityRs['lenOfGroupsUM']['UM'],
                                                                              anonymityRs['lenOfGroupsAfter']['SG'],
                                                                              anonymityRs['lenOfGroupsAfter']['UG'],
                                                                              len(R_inital), len(
                                                                                  RInitalNew),
                                                                              lrp, nrp, drp, cavg)])

    # raise


---------------------------
K_ANONYMITY= 7
Start MM3ar lenTotal 30162
RInitial: 150 RCare: 148
RInitialLow: 50 RCareLow: 34
Begin with  {'SG': 834, 'SGTotal': 19466, 'UG': 6811, 'UGTotal': 10696, 'UM': 0, 'UMTotal': 0}
After migrate  {'SG': 2362, 'SGTotal': 30162, 'UG': 0, 'UGTotal': 0, 'UM': 0, 'UMTotal': 0}
After disperse  {'SG': 2362, 'SGTotal': 30162, 'UG': 0, 'UGTotal': 0, 'UM': 0, 'UMTotal': 0}
Done, Total time: 4509.29s
RInitalNew: 122
LRP: 0.19 NRP: 0.0 DRP:0.19 CAVG: 1.82
---------------------------
K_ANONYMITY= 11
Start MM3ar lenTotal 30162
RInitial: 150 RCare: 148
RInitialLow: 50 RCareLow: 34
Begin with  {'SG': 496, 'SGTotal': 16696, 'UG': 7149, 'UGTotal': 13466, 'UM': 0, 'UMTotal': 0}
After migrate  {'SG': 1725, 'SGTotal': 30162, 'UG': 0, 'UGTotal': 0, 'UM': 0, 'UMTotal': 0}
After disperse  {'SG': 1725, 'SGTotal': 30162, 'UG': 0, 'UGTotal': 0, 'UM': 0, 'UMTotal': 0}
Done, Total time: 5578.37s
RInitalNew: 122
LRP: 0.19 NRP: 0.0 DRP:0.19 CAVG: 1.59
---------------------------